In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip install 'keras-applications'
!pip install 'efficientnet'

     |████████████████████████████████| 50 kB 2.5 MB/s 


In [3]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras import models, Sequential, Input, layers, optimizers, regularizers, callbacks, losses, metrics 
from matplotlib import pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import StratifiedKFold

import efficientnet.tfkeras as effnet
from tensorflow.keras.layers import Input
import warnings
warnings.filterwarnings("ignore")

%matplotlib inline

In [4]:
# FOR KAGGLE
train_images_dir = '../input/petfinder-pawpularity-score/train/'
test_images_dir = '../input/petfinder-pawpularity-score/test/'

# FOR LOCAL PC
# train_images_dir = './dataset/train/'
# test_images_dir = './dataset/test/'

In [5]:
train_df = pd.read_csv('../input/petfinder-pawpularity-score/train.csv')
train_df.loc[:, 'filename'] = train_images_dir + train_df['Id'] + '.jpg'

test_df = pd.read_csv('../input/petfinder-pawpularity-score/test.csv')
test_df.loc[:, 'filename'] = test_images_dir + test_df['Id'] + '.jpg'


# Set a specific label to be able to perform stratification
train_df['stratify_label'] = pd.qcut(train_df['Pawpularity'], q = 30, labels = range(30))
# Label value to be used for feature model 'classification' training.
train_df['target_value'] = train_df['Pawpularity'] / 100.

In [6]:
train_df.head(5)

,Id,Subject Focus,Eyes,Face,Near,Action,Accessory,Group,Collage,Human,Occlusion,Info,Blur,Pawpularity,filename,stratify_label,target_value
0,0007de18844b0dbbb5e1f607da0606e0,0,1,1,1,0,0,1,0,0,0,0,0,63,../input/petfinder-pawpularity-score/train/000...,26,0.63
1,0009c66b9439883ba2750fb825e1d7db,0,1,1,0,0,0,0,0,0,0,0,0,42,../input/petfinder-pawpularity-score/train/000...,20,0.42
2,0013fd999caf9a3efe1352ca1b0d937e,0,1,1,1,0,0,0,0,1,1,0,0,28,../input/petfinder-pawpularity-score/train/001...,9,0.28
3,0018df346ac9c1d8413cfcc888ca8246,0,1,1,1,0,0,0,0,0,0,0,0,15,../input/petfinder-pawpularity-score/train/001...,2,0.15
4,001dc955e10590d3ca4673f034feeef2,0,0,0,1,0,0,1,0,0,0,0,0,72,../input/petfinder-pawpularity-score/train/001...,27,0.72


In [7]:
img_size = (384, 384, 3)
BATCH_SIZE = 32
EPOCHS = 10
LEARNING_RATE = 0.000005
FEATURE_FOLDS = 10
SEED = 42

In [8]:
def augment_image(image, label=None):
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)
    image = tf.image.random_saturation(image, 0.95, 1.05)
    image = tf.image.random_brightness(image, 0.05)
    image = tf.image.random_contrast(image, 0.95, 1.05)
    image = tf.image.random_hue(image, 0.05)
    return image, label

def parse_image(filename, label=None):
    image_string = tf.io.read_file(filename)
    image = tf.image.decode_jpeg(image_string, channels=img_size[2])
    image = tf.image.convert_image_dtype(image, tf.float32)
    image = tf.cast(image, tf.float32) / 255.0
    image = tf.image.resize(image, img_size[:2])
    return image, label

def get_dataset(filenames, label=None, shuffle=False, augment=False, repeat=False):
    dataset = tf.data.TFRecordDataset.from_tensor_slices((filenames, label))     
    dataset = dataset.map(parse_image, num_parallel_calls=4)
    if augment:
        dataset = dataset.map(augment_image, num_parallel_calls=4)
    if repeat:
        dataset = dataset.repeat()
    if shuffle:
        dataset = dataset.shuffle(1024, reshuffle_each_iteration = True)
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.prefetch(True)
    return dataset



In [9]:
def save_checkpoint(fold):
    return tf.keras.callbacks.ModelCheckpoint(f'/kaggle/working/feature_model_{fold}.h5',
                                              monitor = 'val_rmse', 
                                              mode = 'min', 
                                              save_weights_only = True,
                                              save_best_only = True)

def build_model(): 
    # Create and Compile Model and show Summary
    model = effnet.EfficientNetB2(include_top = False, 
                                  classes = None, 
                                  input_shape = img_size, 
                                  weights = 'noisy-student', 
                                  pooling = 'avg')

    # Set all layers to Trainable except BN layers
    for layer in model.layers:
        if isinstance(layer, tf.keras.layers.BatchNormalization):
            layer.trainable = False
        else:
            layer.trainable = True

    
    X = tf.keras.layers.Dropout(0.25)(model.output)
    output = tf.keras.layers.Dense(1, activation = 'sigmoid')(X)
    model = tf.keras.Model(inputs = model.input, outputs = output)
    model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = LEARNING_RATE), 
                  loss = tf.keras.losses.BinaryCrossentropy(), 
                  metrics = [tf.keras.metrics.RootMeanSquaredError('rmse')])        
    
    return model    

In [10]:
all_val_rmse = []

# Stratified Training
kfold = StratifiedKFold(n_splits = FEATURE_FOLDS, shuffle = True, random_state = SEED)
for fold, (train_index, val_index) in enumerate(kfold.split(train_df.index, train_df['stratify_label'])):
    
    tf.keras.backend.clear_session()
    model = build_model()

    train_dataset = get_dataset(train_df.loc[train_index, 'filename'], train_df.loc[train_index, 'target_value'], shuffle=True, augment=True, repeat=True)
    val_dataset = get_dataset(train_df.loc[train_index, 'filename'], train_df.loc[train_index, 'target_value'], repeat=True)
    
    # Fit Model
    history = model.fit(train_dataset,
                        epochs = EPOCHS,
                        steps_per_epoch = train_index.shape[0] // BATCH_SIZE,
                        validation_steps = val_index.shape[0] // BATCH_SIZE,
                        callbacks = [save_checkpoint(fold)],
                        validation_data = val_dataset
                       )

    # Validation Information
    best_val_rmse = min(history.history['val_rmse'])
    all_val_rmse.append(best_val_rmse)
    print(f'\nValidation RMSE: {best_val_rmse}\n')
print(f'\nMean Validation RMSE: {np.mean(all_val_rmse)}\n')


2021-12-18 01:53:54.829833: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-18 01:53:54.968342: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-18 01:53:54.969064: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-18 01:53:54.970213: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

31793152/31782304 [==============================] - 0s 0us/step
Epoch 1/10


2021-12-18 01:54:06.677279: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2021-12-18 01:54:19.626193: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:175] Filling up shuffle buffer (this may take a while): 850 of 1024
2021-12-18 01:54:21.539012: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:228] Shuffle buffer filled.
2021-12-18 01:54:22.890277: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005
2021-12-18 01:54:28.245075: W tensorflow/core/common_runtime/bfc_allocator.cc:272] Allocator (GPU_0_bfc) ran out of memory trying to allocate 1.04GiB with freed_by_count=0. The caller indicates that this is not a failure, but may mean that there could be performance gains if more memory were available.
2021-12-18 01:54:28.255193: W tensorflow/core/common_runtime/bfc_allocator.cc:272] Allocator (GPU_0_bfc) ran out of memory trying to allocate 1.64GiB with freed_by_count=0. The 

278/278 [==============================] - 313s 1s/step - loss: 0.6667 - rmse: 0.2084 - val_loss: 0.6614 - val_rmse: 0.2009
Epoch 2/10
278/278 [==============================] - 286s 1s/step - loss: 0.6662 - rmse: 0.2088 - val_loss: 0.6614 - val_rmse: 0.2010
Epoch 3/10
278/278 [==============================] - 281s 1s/step - loss: 0.6660 - rmse: 0.2076 - val_loss: 0.6616 - val_rmse: 0.2012
Epoch 4/10
278/278 [==============================] - 282s 1s/step - loss: 0.6650 - rmse: 0.2068 - val_loss: 0.6616 - val_rmse: 0.2011
Epoch 5/10
278/278 [==============================] - 286s 1s/step - loss: 0.6656 - rmse: 0.2080 - val_loss: 0.6615 - val_rmse: 0.2010
Epoch 6/10
278/278 [==============================] - 285s 1s/step - loss: 0.6647 - rmse: 0.2067 - val_loss: 0.6618 - val_rmse: 0.2014
Epoch 7/10
278/278 [==============================] - 285s 1s/step - loss: 0.6655 - rmse: 0.2065 - val_loss: 0.6615 - val_rmse: 0.2011
Epoch 8/10
278/278 [==============================] - 286s 1s/step

2021-12-18 02:42:14.363774: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:175] Filling up shuffle buffer (this may take a while): 996 of 1024
2021-12-18 02:42:14.613724: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:228] Shuffle buffer filled.


...

278/278 [==============================] - 308s 1s/step - loss: 0.6674 - rmse: 0.2098 - val_loss: 0.6622 - val_rmse: 0.1996
Epoch 2/10
278/278 [==============================] - 284s 1s/step - loss: 0.6654 - rmse: 0.2076 - val_loss: 0.6624 - val_rmse: 0.1998
Epoch 3/10
278/278 [==============================] - 285s 1s/step - loss: 0.6654 - rmse: 0.2063 - val_loss: 0.6623 - val_rmse: 0.1996
Epoch 4/10
278/278 [==============================] - 285s 1s/step - loss: 0.6659 - rmse: 0.2079 - val_loss: 0.6622 - val_rmse: 0.1996
Epoch 5/10
278/278 [==============================] - 287s 1s/step - loss: 0.6645 - rmse: 0.2057 - val_loss: 0.6632 - val_rmse: 0.2007
Epoch 6/10
278/278 [==============================] - 284s 1s/step - loss: 0.6666 - rmse: 0.2090 - val_loss: 0.6622 - val_rmse: 0.1996
Epoch 7/10
278/278 [==============================] - 285s 1s/step - loss: 0.6645 - rmse: 0.2058 - val_loss: 0.6624 - val_rmse: 0.1998
Epoch 8/10
278/278 [==============================] - 286s 1s/step

2021-12-18 08:19:18.969083: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:175] Filling up shuffle buffer (this may take a while): 1015 of 1024
2021-12-18 08:19:19.081090: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:228] Shuffle buffer filled.


278/278 [==============================] - 306s 1s/step - loss: 0.6669 - rmse: 0.2088 - val_loss: 0.6627 - val_rmse: 0.2048
Epoch 2/10
278/278 [==============================] - 290s 1s/step - loss: 0.6656 - rmse: 0.2077 - val_loss: 0.6625 - val_rmse: 0.2046
Epoch 3/10
278/278 [==============================] - 285s 1s/step - loss: 0.6661 - rmse: 0.2075 - val_loss: 0.6629 - val_rmse: 0.2051
Epoch 4/10
278/278 [==============================] - 285s 1s/step - loss: 0.6654 - rmse: 0.2068 - val_loss: 0.6625 - val_rmse: 0.2046
Epoch 5/10
278/278 [==============================] - 284s 1s/step - loss: 0.6655 - rmse: 0.2080 - val_loss: 0.6625 - val_rmse: 0.2046
Epoch 6/10
278/278 [==============================] - 284s 1s/step - loss: 0.6655 - rmse: 0.2072 - val_loss: 0.6625 - val_rmse: 0.2046
Epoch 7/10
278/278 [==============================] - 285s 1s/step - loss: 0.6649 - rmse: 0.2068 - val_loss: 0.6626 - val_rmse: 0.2047
Epoch 8/10
278/278 [==============================] - 283s 1s/step

2021-12-18 09:07:52.628425: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:175] Filling up shuffle buffer (this may take a while): 890 of 1024
2021-12-18 09:07:54.981005: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:228] Shuffle buffer filled.


278/278 [==============================] - 313s 1s/step - loss: 0.6669 - rmse: 0.2086 - val_loss: 0.6604 - val_rmse: 0.1986
Epoch 2/10
278/278 [==============================] - 283s 1s/step - loss: 0.6655 - rmse: 0.2074 - val_loss: 0.6604 - val_rmse: 0.1986
Epoch 3/10
278/278 [==============================] - 283s 1s/step - loss: 0.6654 - rmse: 0.2071 - val_loss: 0.6604 - val_rmse: 0.1986
Epoch 4/10
278/278 [==============================] - 281s 1s/step - loss: 0.6660 - rmse: 0.2084 - val_loss: 0.6608 - val_rmse: 0.1990
Epoch 5/10
278/278 [==============================] - 281s 1s/step - loss: 0.6655 - rmse: 0.2079 - val_loss: 0.6612 - val_rmse: 0.1995
Epoch 6/10
278/278 [==============================] - 284s 1s/step - loss: 0.6656 - rmse: 0.2071 - val_loss: 0.6607 - val_rmse: 0.1989
Epoch 7/10
278/278 [==============================] - 283s 1s/step - loss: 0.6651 - rmse: 0.2066 - val_loss: 0.6604 - val_rmse: 0.1986
Epoch 8/10
278/278 [==============================] - 282s 1s/step